In [ ]:
!pip install hazm faiss-cpu tqdm

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of faiss-cpu to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 13.2 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9300 sha256=eb9d785a85

## Import libraries

In [ ]:
import faiss
import numpy as np
import pickle
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from google.colab import files
import json
from hazm import Normalizer, WordTokenizer, Stemmer, Lemmatizer, stopwords_list
from tqdm import tqdm

## Upload dataset

In [ ]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving bbcpersian.json to bbcpersian (1).json


## Read dataset and extract titles and bodies

In [ ]:
with open(filename, 'r', encoding='utf-8') as f:
    data = json.load(f)

# extract Titles and Bodies
titles = [item['title'] for item in data]
bodies = [item['body'] for item in data]

## Define Normalizer, Tokenizer, Stemmer, Lemmatizer, stop words

In [ ]:
normalizer = Normalizer()
tokenizer = WordTokenizer()
stemmer = Stemmer()
lemmatizer = Lemmatizer()
stop_words = set(stopwords_list())

 19%|█▉        | 723/3780 [1:14:45<5:16:07,  6.20s/it]


## Preprocessing

In [ ]:
def preprocess(text):
    text = normalizer.normalize(text)
    tokens = tokenizer.tokenize(text)
    tokens = [t for t in tokens if t not in stop_words]
    tokens = [lemmatizer.lemmatize(stemmer.stem(t)) for t in tokens]
    return " ".join(tokens)

## Preprocess documents, apply vectorization and indexing

In [ ]:
documents = [preprocess(text) for text in tqdm(bodies)]

vectorizer = TfidfVectorizer()
doc_vectors = vectorizer.fit_transform(documents)
index = faiss.IndexFlatIP(doc_vectors.shape[1])
index.add(doc_vectors.toarray())

100%|██████████| 3780/3780 [00:44<00:00, 84.45it/s]


In [ ]:
FEEDBACK_FILE = "feedback_memory.pkl"

if os.path.exists(FEEDBACK_FILE):
    with open(FEEDBACK_FILE, "rb") as f:
        feedback_memory = pickle.load(f)
else:
    feedback_memory = {}


In [ ]:
def apply_rocchio(query_vec, relevant_vecs, nonrelevant_vecs, alpha=1.0, beta=0.75, gamma=0.25):
    if len(relevant_vecs) > 0:
        relevant_centroid = np.mean(relevant_vecs, axis=0)
    else:
        relevant_centroid = 0

    if len(nonrelevant_vecs) > 0:
        nonrelevant_centroid = np.mean(nonrelevant_vecs, axis=0)
    else:
        nonrelevant_centroid = 0

    updated_vec = alpha * query_vec + beta * relevant_centroid - gamma * nonrelevant_centroid
    return normalize(updated_vec.reshape(1, -1))[0]

In [ ]:
def get_user_feedback(results):
    print("\nInitial results:")
    for idx, (real_index, score) in enumerate(results):
        score_percent = round(score * 100, 2)
        print(f"[{idx}] Title: {titles[real_index]} ({score_percent}%)\n→ Body:{documents[real_index][:60]}...\n")

    input_str = input("Enter relevant result number (e.g. 1 3): ")
    relevant_indices = list(map(int, input_str.strip().split()))

    relevant_vecs, nonrelevant_vecs = [], []

    for idx, (real_index, score) in enumerate(results):
        vec = doc_vectors[real_index].toarray().astype(np.float32)[0]
        if idx in relevant_indices:
            relevant_vecs.append(vec)
        else:
            nonrelevant_vecs.append(vec)

    return relevant_vecs, nonrelevant_vecs

In [ ]:
def run_feedback_search():
    query = input("Your query: ").strip()

    if query in feedback_memory:
        query_vec = feedback_memory[query]
        print("Using previous vector of this query...")
        D, I = index.search(query_vec.reshape(1, -1), 5)
        results = list(zip(I[0], D[0]))
        for idx, (real_index, score) in enumerate(results):
          score_percent = round(score * 100, 2)
          print(f"[{idx}] Title: {titles[real_index]} ({score_percent}%)\n→ Body: {documents[real_index][:60]}...\n")
        return

    else:
        query_vec = vectorizer.transform([query]).toarray().astype(np.float32)
        query_vec = normalize(query_vec)[0]

    # search
    D, I = index.search(query_vec.reshape(1, -1), 5)
    results = list(zip(I[0], D[0]))

    # show results and collect feedback
    relevant_vecs, nonrelevant_vecs = get_user_feedback(results)

    # update with Rocchio
    updated_vec = apply_rocchio(query_vec, relevant_vecs, nonrelevant_vecs)

    # save updated vector
    feedback_memory[query] = updated_vec
    with open(FEEDBACK_FILE, "wb") as f:
        pickle.dump(feedback_memory, f)

    print("Model updated.")


## Example 1

In [ ]:
run_feedback_search()

Your query: قرارداد

Initial results:
[0] Title: سند همکاری ایران و چین؛ 'شر در جزئیات نهفته است' (44.59%)
→ Body:سیاو اردلانبی‌بی‌س قضاو قرارداد ۲۵ ایر چین نامیده خارج چارچو...

[1] Title: کرسنت؛ غرامت '۶۰۰ میلیون دلاری' برای شرکت اماراتی در پرونده شکایت علیه ایران (38.77%)
→ Body:شرک « دانا گاز » امار اعلا هیئ داور بین‌الملل پرونده شکا شرک...

[2] Title: رابطه ایران و چین، موضوع کشمکش تازه سیاسی در ایران (37.22%)
→ Body:فایل لطفا جاوا اسکریپ فعال مرورگر استفاده . رابطه ایر چین ، ...

[3] Title: تابستان جذاب فوتبال اروپا؛ ستاره‌هایی که آزاد می‌شوند (28.99%)
→ Body:باشگاه دوس بازیکن علاقه ارزان قیم بخرد ، این‌که دلیل شیوع کر...

[4] Title: کارگری خود را کنار چاه نفت هویزه کشت (27.28%)
→ Body:فایل لطفا جاوا اسکریپ فعال مرورگر استفاده . کارگر فعال مید ن...

Enter relevant result number (e.g. 1 3): 0 2
Model updated.


In [ ]:
run_feedback_search()

Your query: قرارداد
Using previous vector of this query...
[0] Title: سند همکاری ایران و چین؛ 'شر در جزئیات نهفته است' (67.67%)
→ Body: سیاو اردلانبی‌بی‌س قضاو قرارداد ۲۵ ایر چین نامیده خارج چارچو...

[1] Title: رابطه ایران و چین، موضوع کشمکش تازه سیاسی در ایران (63.05%)
→ Body: فایل لطفا جاوا اسکریپ فعال مرورگر استفاده . رابطه ایر چین ، ...

[2] Title: دولت حامی و منتقدان عصبانی؛ از توافق ۲۵ ساله ایران و چین چه درز کرده؟ (38.31%)
→ Body: جنجال توافق ۲۵ ایر چین ادامه . دول تاکید موضوع مخف نمایندگ م...

[3] Title: سند همکاری‌ ایران و چین؛ 'حکمت‌آمیز' یا 'قرارداد ترکمن‌چای' (35.48%)
→ Body: برنامه بلندمد همکاری ایر چین دول حسن روحان « سند راهبرد » مخ...

[4] Title: کرسنت؛ غرامت '۶۰۰ میلیون دلاری' برای شرکت اماراتی در پرونده شکایت علیه ایران (34.14%)
→ Body: شرک « دانا گاز » امار اعلا هیئ داور بین‌الملل پرونده شکا شرک...



## Example 2

In [ ]:
run_feedback_search()

Your query: فوتبال انگلیس

Initial results:
[0] Title: پیش از کرونا چه چیزهایی فوتبال انگلیس را تعطیل کرد؟ (48.89%)
→ Body:تماشاگر فوتبال انگلیس نقاط جه مشکل همه‌گیر جهان ویروس کرونا ...

[1] Title: کرونا دنیای فوتبال دوستان را تیره و تار کرد؛ فوتبال انگلستان، آلمان و فرانسه تعلیق شد (41.67%)
→ Body:فایل لطفا جاوا اسکریپ فعال مرورگر استفاده . کرونا دنیا فوتبا...

[2] Title: نیمه‌نهایی یورو ۲۰۲۰: انگلیس با پیروزی مقابل دانمارک حریف ایتالیا در فینال شد (38.78%)
→ Body:ت مل فوتبال انگلیس ۵۵ سال انتظار توانس فینال تورنمن ( جا جها...

[3] Title: فینال یورو ۲۰۲۰؛ آرزوی بزرگ انگلیس و ایتالیا (38.44%)
→ Body:پویا عنایتیخبرنگار دیدار #هست تورنمنت شیوع کرونا برگزار سال ...

[4] Title: یورو ۲۰۲۰؛ مسابقه با دانمارک پس از شب رویایی انگلیس در رم (35.06%)
→ Body:ت مل فوتبال انگلیس بازی تاریخ اوکراین گل شهر ر شکس . ت چهارش...

Enter relevant result number (e.g. 1 3): 2 3 4
Model updated.


In [ ]:
run_feedback_search()

Your query: فوتبال انگلیس
Using previous vector of this query...
[0] Title: نیمه‌نهایی یورو ۲۰۲۰: انگلیس با پیروزی مقابل دانمارک حریف ایتالیا در فینال شد (69.24%)
→ Body: ت مل فوتبال انگلیس ۵۵ سال انتظار توانس فینال تورنمن ( جا جها...

[1] Title: فینال یورو ۲۰۲۰؛ آرزوی بزرگ انگلیس و ایتالیا (67.46%)
→ Body: پویا عنایتیخبرنگار دیدار #هست تورنمنت شیوع کرونا برگزار سال ...

[2] Title: یورو ۲۰۲۰؛ مسابقه با دانمارک پس از شب رویایی انگلیس در رم (65.49%)
→ Body: ت مل فوتبال انگلیس بازی تاریخ اوکراین گل شهر ر شکس . ت چهارش...

[3] Title: نیمه‌نهایی یورو ۲۰۲۰؛ انگلیس-دانمارک: رویای ۶۶ یا رویای ۹۲؟ (57.04%)
→ Body: غافلگیرکننده تیم یورو ۲۰۲۰ ؛ انگلیس دانمارک باز نیمه‌ن یورو ...

[4] Title: قهرمانی ایتالیا در یورو ۲۰۲۰؛ مانچینی روی ابرها (46.9%)
→ Body: پویا عنایتیخبرنگار پا تورنمن باشکوه یورو ۲۰۲۰ قهرمان ایتالیا...

